In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

In [ ]:
train_dir = 'C:/ML/DataSetTrain'
valid_dir = 'C:/ML/DataSetValid'
test_dir = 'C:/ML/DataSetTest'

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 64
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

In [ ]:
datagen_train = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    zoom_range = 0.25,
    rotation_range = 30,
    width_shift_range = 0.25,
    height_shift_range = 0.25,
    horizontal_flip = True,
    vertical_flip = True)

train_generator = datagen_train.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE)

datagen_valid = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255)

valid_generator = datagen_valid.flow_from_directory(
    valid_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE)

datagen_test = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255)

test_generator = datagen_test.flow_from_directory(
    test_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE)

In [ ]:
mnv2_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               weights='imagenet', 
                                               include_top=False)

In [ ]:
mnv2_model.trainable = False

In [ ]:
new_model = tf.keras.Sequential([
    mnv2_model,
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
new_model.summary()

In [ ]:
new_model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

In [ ]:
epochs = 25

history = new_model.fit(train_generator, 
                    steps_per_epoch=len(train_generator), 
                    epochs=epochs,
                    validation_data=valid_generator, 
                    validation_steps=len(valid_generator)
                    )

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
new_model.evaluate(test_generator)

In [ ]:
new_model.save('mnv2_model_new.h5')

In [ ]:
new_model = tf.keras.models.load_model('mnv2_model_new.h5')